## 수정사항
#### 2021.06.04
* Gripper 자유도 45도로 제한됨에 따라 파지점 방향 수정, 바닥 파지면은 삭제

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

ALG_DATA_PATH = os.path.join(DATA_PATH, "latticized")
try_mkdir(ALG_DATA_PATH)

In [3]:
VISUALIZE = False
TIMEOUT_REACH = 30
TIMEOUT_RETRIEVE = 30
TIMEOUT_SELF = 1
CLEARANCE = 1e-3

ROBOT_TYPE = RobotType.indy7gripper
ROBOT_NAME = "indy0"
TOOL_LINK = "indy0_tcp"
TOOL_XYZ = (0,0,0.14)
TOOL_RPY = (-np.pi/2,0,0)
GRIP_DEPTH = 0.05
HOME_POSE = (0,0,0,0,0,0)

# ROBOT_TYPE = RobotType.panda
# ROBOT_NAME = "panda0"
# TOOL_LINK = "panda0_hand"
# TOOL_XYZ = (0,0,0.112)
# TOOL_RPY = (-np.pi/2,0,0)
# GRIP_DEPTH = 0.03
# HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

connection command:
indy0: False


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0))

## planner

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan = MoveitPlanner(pscene)
checker = MoveitPlanner(pscene)

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checker.motion_filters = [gcheck] # rchecker is not perfect, don't use for data gen

## ui

In [12]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Object Classes

In [13]:
from pkg.utils.gjk import get_point_list, get_gjk_distance
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask
from pkg.planning.filtering.lattice_model.scene_building import *
        
WORKPLANE_TYPES = [WorkPlane, Box, SideBox, TopBox]

Nmax_wall, Nmax_pole, Nmax_bar = 4, 8, 8

Nmax_obj = 3

In [14]:
from pkg.planning.filtering.lattice_model.latticizer_py import *
import itertools

ltc_full = Latticizer_py(WDH=(3, 3, 3), L_CELL=0.05, OFFSET_ZERO=(1.5, 1.5, 1.5))
ltc_effector = Latticizer_py(WDH=(1, 1, 1), L_CELL=0.05, OFFSET_ZERO=(0.5, 0.5, 0.5))
ltc_arm_05 = Latticizer_py(WDH=(2, 2, 2), L_CELL=0.05, OFFSET_ZERO=(0.5, 1.0, 1.0))
ltc_arm_10 = Latticizer_py(WDH=(2, 2, 2), L_CELL=0.10, OFFSET_ZERO=(0.5, 1.0, 1.0))

### sampling

In [15]:
gtimer = GlobalTimer.instance()
gtimer.reset()

reach_success_list = []
reach_time_list = []
retrieve_success_list = []
retrieve_time_list = []

for _ in range(15):
    ROBOT_DATA_ROOT = os.path.join(ALG_DATA_PATH, ROBOT_TYPE.name+"-failmore")
    try_mkdir(ROBOT_DATA_ROOT)

    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
    try_mkdir(DATASET_PATH)
    print("")
    print("DATASET_PATH: {}".format(DATASET_PATH))
    print("")

    GRASP_PATH = os.path.join(DATASET_PATH, "grasp")
    try_mkdir(GRASP_PATH)

    ARM_10_PATH = os.path.join(DATASET_PATH, "arm_10")
    try_mkdir(ARM_10_PATH)

    ARM_05_PATH = os.path.join(DATASET_PATH, "arm_05")
    try_mkdir(ARM_05_PATH)

    FULL_SCENE_PATH = os.path.join(DATASET_PATH, "full_scene")
    try_mkdir(FULL_SCENE_PATH)

    GSCENE_PATH = os.path.join(DATASET_PATH, "gscene")
    try_mkdir(GSCENE_PATH)

    N_max_sample = 1000
    N_print = 5

    gtimer.tic("full_loop")

    i_s = 0
    i_print = 0
    reach_list = []
    retrieve_list = []
    while i_s < N_max_sample:
        ## add floor, ceiling
        floor = Floor(gscene, "floor")
        # ceiling = Ceiling(gscene, "ceiling")

        ## set workplane
        wp = random.choice(WORKPLANE_TYPES)(gscene, "wp", floor_height=floor.RTH[2]+floor.DIM[2]/2)
        pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

        ## add walls, poles, bars
        walls = redistribute_class(gscene, Wall, "wl", Nmax_wall, workplane_avoid=wp)
        poles = redistribute_class(gscene, Pole, "po", Nmax_pole, workplane_avoid=wp)
        bars = redistribute_class(gscene, Bar, "bar", Nmax_bar, workplane_avoid=wp)

        ## add object
        obj_list = disperse_objects(gscene, PlaneObject, "obj", Nmax_obj, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

        samples = []
        for obj in obj_list:
            SHOW_PERIOD = 0.01
            N_sample_single_obj = 3
            N_sample_single_obj_max = 3
            obj_pscene, handles = add_object(pscene, obj)
            mplan.update_gscene()
            initial_state = pscene.initialize_state(HOME_POSE)
            pscene.set_object_state(initial_state)
            from_state = initial_state.copy(pscene)
            to_node = ("grip0",)
            available_binding_dict = pscene.get_available_binding_dict(from_state, to_node, HOME_DICT)
            samples_obj = []
            for _ in range(N_sample_single_obj):
                to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
                
                pscene.set_object_state(from_state)
                
                Traj_reach, LastQ, error, res, binding_list = \
                            checker.plan_transition(from_state=from_state, to_state=to_state, only_self_collision=True, timeout=TIMEOUT_SELF)

                success_reach, success_retrieve = False, False
                time_reach, time_retrieve = TIMEOUT_REACH, TIMEOUT_RETRIEVE
                if res:
                    gtimer.tic("reach")
                    Traj_reach, LastQ, error, success_reach, binding_list = mplan.plan_transition(
                        from_state=from_state, to_state=to_state, timeout=TIMEOUT_REACH)
                    time_reach = gtimer.toc("reach")
                    reach_success_list.append(success_reach)
                    reach_time_list.append(time_reach)
        #             print("reach: {}".format(success_reach))
                    Traj_retrieve = []
                    if success_reach:
                        if VISUALIZE:
                            gscene.show_motion(Traj_reach, period=SHOW_PERIOD)
                        for bd in binding_list:
                            pscene.rebind(bd, list2dict(LastQ, pscene.gscene.joint_names))
                        binding_state, state_param = pscene.get_object_state()
                        new_state = State(binding_state, state_param, list(LastQ), pscene)
                        end_state = new_state.copy(pscene)
                        end_state.Q = np.array(HOME_POSE)
                        gtimer.tic("retrieve")
                        Traj_retrieve, LastQ, error, success_retrieve, binding_list = mplan.plan_transition(
                            from_state=new_state, to_state=end_state, timeout=TIMEOUT_RETRIEVE)
                        time_retrieve = gtimer.toc("retrieve")
                        retrieve_success_list.append(success_retrieve)
                        retrieve_time_list.append(time_retrieve)
        #                 print("retrieve: {}".format(success_retrieve))
                        if success_retrieve and VISUALIZE:
                            gscene.show_motion(Traj_retrieve, period=SHOW_PERIOD)
                    samples_obj.append((obj.name, from_state, to_state, 
                                        success_reach, success_retrieve, Traj_reach, Traj_retrieve, time_reach, time_retrieve))
                    if len(samples_obj)>=N_sample_single_obj_max:
                        pscene.set_object_state(initial_state)
                        break
                else:
                    Traj_reach, Traj_retrieve = [], []
                    reach_success_list.append(success_reach)
                    reach_time_list.append(time_reach)
                    retrieve_success_list.append(success_retrieve)
                    retrieve_time_list.append(time_retrieve)
                    samples_obj.append((obj.name, from_state, to_state, 
                                        success_reach, success_retrieve, Traj_reach, Traj_retrieve, time_reach, time_retrieve))
#                 if success_retrieve and not res_col:
#                     raise(RuntimeError("Something's wrong: successful motion but collision checker failed"))
        #             print("no save?")
            samples = samples+samples_obj
            pscene.set_object_state(initial_state)

            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)    

            gscene.update_markers_all()

        for sample in samples:
            obj_name, from_state, to_state, success_reach, success_retrieve, \
                Traj_reach, Traj_retrieve, time_reach, time_retrieve = sample
            obj = [obj for obj in obj_list if obj.name == obj_name][0]
            obj_pscene, handles = add_object(pscene, obj)
            
                    
            gtem_args = []
            for gtem in gscene:
                if gtem.link_name == "base_link":
                    gtem_args.append(deepcopy(gtem.get_args()))
            obj_args = {obj.name: {"DIM": obj.DIM, "gname": obj.geometry.name} 
                        for obj in obj_list}
        
            pscene.set_object_state(from_state)
            subject_list, binding_ok = pscene.get_changing_subjects(from_state, to_state)
            assert binding_ok, "no available transition"
            assert len(subject_list) == 1, "multiple binding transition - only single allowed in training process"
            sname = subject_list[0]

            grasp_dict = {}
            arm_05_dict = {}
            arm_10_dict = {}
            full_scene_dict = {}
            ltc_effector.clear()
            ltc_arm_05.clear()
            ltc_arm_10.clear()
            ltc_full.clear()

            btf = to_state.binding_state[sname]
            obj_name, ap_name, binder_name, binder_geometry_name = btf.get_chain()
            actor, obj = pscene.actor_dict[binder_name], pscene.subject_dict[obj_name]
            handle = obj.action_points_dict[ap_name]
            T_loal = btf.T_loal

            (object_geo_list, object_T2end_dict), (actor_geo_list, actor_T2end_dict) = \
                            gcheck.get_geolist_tflist_pairs(actor, obj, HOME_DICT)

            (object_geo_list, object_T2end_dict), (actor_geo_list, actor_T2end_dict) = \
                            gcheck.get_geolist_tflist_pairs(actor, obj, HOME_DICT)


            obj_names = obj.geometry.get_family()

            group_name = ROBOT_NAME
            T_br = gscene.get_tf(crob.get_robot_base_dict()[group_name], Q=crob.home_dict,
                                 from_link=obj.geometry.link_name)
            T_re = np.matmul(SE3_inv(T_br), T_loal)
            T_tool_to_rob = T_re
            tool_geo_list, tool_T2end_dict = actor_geo_list, actor_T2end_dict
            T_tar_to_rob = SE3_inv(T_br)
            target_geo_list, target_T2end_dict = object_geo_list, object_T2end_dict

            grasp_dict = {}
            arm_05_dict = {}
            arm_10_dict = {}
            full_scene_dict = {}
            ltc_effector.clear()
            ltc_arm_05.clear()
            ltc_arm_10.clear()
            ltc_full.clear()

            r, th, h = cart2cyl(*T_tool_to_rob[:3, 3])
            T_rl = SE3(Rot_axis(3, th), T_re[:3, 3])  # in robot base link coordinate
            target_names = [gtem.name for gtem in target_geo_list if gtem.name not in obj_names]
            tool_names = [gtem.name for gtem in tool_geo_list]

            ## Convert effector
            T_gl_list = []
            gtem_list = target_geo_list + tool_geo_list
            for gtem in gtem_list:
                if gtem.link_name in tool_T2end_dict:
                    T_rg = matmul_series(T_tool_to_rob, tool_T2end_dict[gtem.link_name], gtem.Toff)
                else:
                    T_rg = matmul_series(T_tar_to_rob, target_T2end_dict[gtem.link_name], gtem.Toff)
                T_lg = np.matmul(SE3_inv(T_rl), T_rg)
                T_gl = SE3_inv(T_lg)
                T_gl_list.append(T_gl)

            ltc_effector.convert_vertices_approx(gtem_list, T_gl_list)

            ## Convert env
            link_env = [lname for lname in gscene.link_names
                        if lname not in pscene.robot_chain_dict[group_name]["link_names"]]
            gtems_env = [gtem for gtem in gscene
                         if gtem.collision and gtem.link_name in link_env]
            Trl_base = SE3(T_rl[:3, :3], (0, 0, shoulder_height))  # in robot base link coordinate
            T_bl = np.matmul(T_br, Trl_base)
            T_lb = SE3_inv(T_bl)
            Tlist_env = {lname: gscene.get_tf(lname, crob.home_dict) for lname in link_env}
            T_gl_list_env = []
            for gtem in gtems_env:
                T_lg = matmul_series(T_lb, Tlist_env[gtem.link_name], gtem.Toff)
                T_gl = SE3_inv(T_lg)
                T_gl_list_env.append(T_gl)
            ltc_arm_05.convert_vertices_approx(gtems_env, T_gl_list_env)
            ltc_arm_10.convert_vertices_approx(gtems_env, T_gl_list_env)

            T_gl_list_tool = []
            for gtem in tool_geo_list:
                T_rg = matmul_series(T_tool_to_rob, tool_T2end_dict[gtem.link_name], gtem.Toff)
                T_lg = matmul_series(T_lb, T_br, T_rg)
                T_gl = SE3_inv(T_lg)
                T_gl_list_tool.append(T_gl)
            ltc_full.convert_vertices_approx(gtems_env, T_gl_list_env)
            ltc_full.convert_vertices_approx(tool_geo_list, T_gl_list_tool)

            grasp_dict["tar"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in target_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["tool"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in tool_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["obj"] = sorted(set(itertools.chain(*[ltc_effector.coll_idx_dict[tname] for tname in obj_names if tname in ltc_effector.coll_idx_dict])))
            grasp_dict["T_end_effector"], grasp_dict["T_end_joint"], grasp_dict["Tref_base"]  = T_loal, T_loal, Trl_base
            grasp_dict["reach"], grasp_dict["retrieve"] = success_reach, success_retrieve
            grasp_dict["reach_time"], grasp_dict["retrieve_time"] = time_reach, success_retrieve

            arm_05_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_05.coll_idx_dict])))
            arm_05_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_05.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_05.coll_idx_dict])))
            arm_05_dict["T_end_effector"], arm_05_dict["T_end_joint"], arm_05_dict["Tref_base"] = T_loal, T_loal, Trl_base
            arm_05_dict["reach"], arm_05_dict["retrieve"] = success_reach, success_retrieve

            arm_10_dict["tar"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in target_names if tname in ltc_arm_10.coll_idx_dict])))
            arm_10_dict["tool"] = sorted(set(itertools.chain(*[ltc_arm_10.coll_idx_dict[tname] for tname in tool_names if tname in ltc_arm_10.coll_idx_dict])))
            arm_10_dict["T_end_effector"], arm_10_dict["T_end_joint"], arm_10_dict["Tref_base"] = T_loal, T_loal, Trl_base
            arm_10_dict["reach"], arm_10_dict["retrieve"] = success_reach, success_retrieve

            full_scene_dict["tar"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in target_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["tool"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in tool_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["obj"] = sorted(set(itertools.chain(*[ltc_full.coll_idx_dict[tname] for tname in obj_names if tname in ltc_full.coll_idx_dict])))
            full_scene_dict["T_end_effector"], full_scene_dict["T_end_joint"], full_scene_dict["Tref_base"] = T_loal, T_loal, Trl_base
            full_scene_dict["reach"], full_scene_dict["retrieve"] = success_reach, success_retrieve

            reach_list.append(success_reach)
            retrieve_list.append(success_retrieve)

            save_pickle(os.path.join(GRASP_PATH, "%06d.pkl"%i_s), grasp_dict)
            save_pickle(os.path.join(ARM_05_PATH, "%06d.pkl"%i_s), arm_05_dict)
            save_pickle(os.path.join(ARM_10_PATH, "%06d.pkl"%i_s), arm_10_dict)
            save_pickle(os.path.join(FULL_SCENE_PATH, "%06d.pkl"%i_s), full_scene_dict)
            save_pickle(os.path.join(GSCENE_PATH, "%06d.pkl"%i_s), 
                        {"gtem_args":gtem_args, "obj_args": obj_args, "T_loal": T_loal})

            i_s += 1

            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)

            gscene.update_markers_all()
        if i_s > 0 :
            if i_s > i_print*N_print:
                i_print +=  1
                print_end = "\n"
            else:
                print_end = "\r"
            time_elapsed = gtimer.toc("full_loop")/1000
            print("{} / {} in {} / {} s -- reach,retrieve = ({} %, {} %)                     ".format(
                i_s, N_max_sample, round(time_elapsed, 2), round(time_elapsed/i_s*N_max_sample, 2), 
                round(np.mean(reach_list)*100, 1), round(np.mean(retrieve_list)*100, 1)), end=print_end)


    print("")
    print("")
    print("============= Finished {} in {} s -- reach,retrieve = ({} %, {} %) =================".format(
        i_s, round(time_elapsed, 2), round(np.mean(reach_list)*100, 1), round(np.mean(retrieve_list)*100, 1)))


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/latticized/indy7gripper-failmore/20211003-203350

3 / 1000 in 2.07 / 688.59 s -- reach,retrieve = (0.0 %, 0.0 %)                     
6 / 1000 in 3.5 / 584.05 s -- reach,retrieve = (0.0 %, 0.0 %)                     
15 / 1000 in 7.79 / 519.63 s -- reach,retrieve = (13.3 %, 13.3 %)                     
21 / 1000 in 13.03 / 620.65 s -- reach,retrieve = (14.3 %, 14.3 %)                     
24 / 1000 in 14.23 / 592.83 s -- reach,retrieve = (12.5 %, 12.5 %)                     
27 / 1000 in 16.24 / 601.65 s -- reach,retrieve = (14.8 %, 14.8 %)                     
36 / 1000 in 20.23 / 562.07 s -- reach,retrieve = (11.1 %, 11.1 %)                     
42 / 1000 in 23.28 / 554.28 s -- reach,retrieve = (9.5 %, 9.5 %)                     
45 / 1000 in 23.91 / 531.36 s -- reach,retrieve = (8.9 %, 8.9 %)                     
51 / 1000 in 26.0 / 509.85 s -- reach,retrieve = (7.8 %, 7.8 %)                     
54 / 1000 in 28.12 / 520.74 s -- re

483 / 1000 in 462.07 / 956.67 s -- reach,retrieve = (5.8 %, 5.8 %)                     
489 / 1000 in 468.06 / 957.18 s -- reach,retrieve = (5.9 %, 5.9 %)                     
492 / 1000 in 469.17 / 953.61 s -- reach,retrieve = (5.9 %, 5.9 %)                     
495 / 1000 in 472.17 / 953.87 s -- reach,retrieve = (5.9 %, 5.9 %)                     
501 / 1000 in 475.84 / 949.77 s -- reach,retrieve = (6.0 %, 6.0 %)                     
507 / 1000 in 479.76 / 946.27 s -- reach,retrieve = (5.9 %, 5.9 %)                     
513 / 1000 in 483.19 / 941.88 s -- reach,retrieve = (5.8 %, 5.8 %)                     
522 / 1000 in 486.23 / 931.48 s -- reach,retrieve = (5.7 %, 5.7 %)                     
528 / 1000 in 489.43 / 926.96 s -- reach,retrieve = (6.1 %, 6.1 %)                     
534 / 1000 in 492.24 / 921.8 s -- reach,retrieve = (6.6 %, 6.6 %)                     
540 / 1000 in 498.74 / 923.59 s -- reach,retrieve = (6.5 %, 6.5 %)                     
549 / 1000 in 507.27 / 924.0 s --

957 / 1000 in 1109.7 / 1159.56 s -- reach,retrieve = (6.7 %, 6.6 %)                     
963 / 1000 in 1114.66 / 1157.48 s -- reach,retrieve = (6.6 %, 6.5 %)                     
969 / 1000 in 1118.65 / 1154.44 s -- reach,retrieve = (6.7 %, 6.6 %)                     
972 / 1000 in 1120.4 / 1152.68 s -- reach,retrieve = (6.7 %, 6.6 %)                     
978 / 1000 in 1123.68 / 1148.96 s -- reach,retrieve = (6.6 %, 6.5 %)                     
984 / 1000 in 1124.77 / 1143.06 s -- reach,retrieve = (6.6 %, 6.5 %)                     
987 / 1000 in 1129.41 / 1144.29 s -- reach,retrieve = (6.6 %, 6.5 %)                     
990 / 1000 in 1160.78 / 1172.51 s -- reach,retrieve = (6.6 %, 6.5 %)                     
993 / 1000 in 1162.21 / 1170.41 s -- reach,retrieve = (6.5 %, 6.4 %)                     
996 / 1000 in 1205.93 / 1210.77 s -- reach,retrieve = (6.6 %, 6.4 %)                     
1002 / 1000 in 1239.26 / 1236.79 s -- reach,retrieve = (6.6 %, 6.4 %)                     


=========

444 / 1000 in 497.9 / 1121.41 s -- reach,retrieve = (6.3 %, 6.3 %)                     
447 / 1000 in 498.73 / 1115.72 s -- reach,retrieve = (6.5 %, 6.5 %)                     
450 / 1000 in 500.45 / 1112.1 s -- reach,retrieve = (6.4 %, 6.4 %)                     
453 / 1000 in 502.05 / 1108.27 s -- reach,retrieve = (6.4 %, 6.4 %)                     
459 / 1000 in 504.75 / 1099.67 s -- reach,retrieve = (6.3 %, 6.3 %)                     
465 / 1000 in 507.21 / 1090.77 s -- reach,retrieve = (6.2 %, 6.2 %)                     
471 / 1000 in 508.96 / 1080.61 s -- reach,retrieve = (6.2 %, 6.2 %)                     
480 / 1000 in 513.62 / 1070.04 s -- reach,retrieve = (6.0 %, 6.0 %)                     
486 / 1000 in 516.3 / 1062.35 s -- reach,retrieve = (6.2 %, 6.2 %)                     
489 / 1000 in 517.41 / 1058.1 s -- reach,retrieve = (6.1 %, 6.1 %)                     
492 / 1000 in 518.06 / 1052.96 s -- reach,retrieve = (6.1 %, 6.1 %)                     
498 / 1000 in 524.18 / 10

912 / 1000 in 987.43 / 1082.71 s -- reach,retrieve = (5.8 %, 5.8 %)                     
918 / 1000 in 990.9 / 1079.41 s -- reach,retrieve = (5.8 %, 5.8 %)                     
921 / 1000 in 992.2 / 1077.31 s -- reach,retrieve = (5.8 %, 5.8 %)                     
927 / 1000 in 1025.6 / 1106.36 s -- reach,retrieve = (5.7 %, 5.7 %)                     
933 / 1000 in 1027.46 / 1101.24 s -- reach,retrieve = (5.7 %, 5.7 %)                     
939 / 1000 in 1028.82 / 1095.66 s -- reach,retrieve = (5.6 %, 5.6 %)                     
942 / 1000 in 1032.9 / 1096.5 s -- reach,retrieve = (5.6 %, 5.6 %)                     
951 / 1000 in 1043.17 / 1096.92 s -- reach,retrieve = (5.6 %, 5.6 %)                     
960 / 1000 in 1050.35 / 1094.11 s -- reach,retrieve = (5.7 %, 5.7 %)                     
969 / 1000 in 1059.73 / 1093.63 s -- reach,retrieve = (5.9 %, 5.9 %)                     
972 / 1000 in 1060.81 / 1091.37 s -- reach,retrieve = (5.9 %, 5.9 %)                     
978 / 1000 in 1063

...

774 / 1000 in 825.59 / 1066.65 s -- reach,retrieve = (6.5 %, 6.3 %)                     
780 / 1000 in 827.65 / 1061.09 s -- reach,retrieve = (6.4 %, 6.3 %)                     
786 / 1000 in 829.37 / 1055.18 s -- reach,retrieve = (6.4 %, 6.2 %)                     
792 / 1000 in 833.76 / 1052.73 s -- reach,retrieve = (6.3 %, 6.2 %)                     
795 / 1000 in 834.63 / 1049.85 s -- reach,retrieve = (6.4 %, 6.3 %)                     
798 / 1000 in 837.26 / 1049.2 s -- reach,retrieve = (6.4 %, 6.3 %)                     
804 / 1000 in 841.67 / 1046.86 s -- reach,retrieve = (6.5 %, 6.3 %)                     
813 / 1000 in 847.21 / 1042.08 s -- reach,retrieve = (6.4 %, 6.3 %)                     
816 / 1000 in 848.51 / 1039.84 s -- reach,retrieve = (6.5 %, 6.4 %)                     
822 / 1000 in 855.26 / 1040.47 s -- reach,retrieve = (6.4 %, 6.3 %)                     
828 / 1000 in 857.94 / 1036.16 s -- reach,retrieve = (6.4 %, 6.3 %)                     
834 / 1000 in 860.12 /

237 / 1000 in 336.17 / 1418.44 s -- reach,retrieve = (6.3 %, 6.3 %)                     
243 / 1000 in 338.77 / 1394.13 s -- reach,retrieve = (6.6 %, 6.6 %)                     
249 / 1000 in 342.0 / 1373.48 s -- reach,retrieve = (6.4 %, 6.4 %)                     
255 / 1000 in 345.09 / 1353.28 s -- reach,retrieve = (6.3 %, 6.3 %)                     
261 / 1000 in 348.69 / 1335.99 s -- reach,retrieve = (6.1 %, 6.1 %)                     
270 / 1000 in 355.04 / 1314.97 s -- reach,retrieve = (6.7 %, 6.7 %)                     
273 / 1000 in 356.25 / 1304.96 s -- reach,retrieve = (6.6 %, 6.6 %)                     
279 / 1000 in 362.11 / 1297.87 s -- reach,retrieve = (6.8 %, 6.8 %)                     
282 / 1000 in 365.2 / 1295.02 s -- reach,retrieve = (6.7 %, 6.7 %)                     
285 / 1000 in 368.19 / 1291.91 s -- reach,retrieve = (6.7 %, 6.7 %)                     
288 / 1000 in 369.96 / 1284.59 s -- reach,retrieve = (6.6 %, 6.6 %)                     
297 / 1000 in 375.25 / 

708 / 1000 in 804.9 / 1136.86 s -- reach,retrieve = (7.9 %, 7.9 %)                     
714 / 1000 in 897.13 / 1256.48 s -- reach,retrieve = (7.8 %, 7.8 %)                     
717 / 1000 in 899.15 / 1254.05 s -- reach,retrieve = (7.8 %, 7.8 %)                     
723 / 1000 in 901.58 / 1247.0 s -- reach,retrieve = (7.7 %, 7.7 %)                     
726 / 1000 in 903.72 / 1244.79 s -- reach,retrieve = (7.7 %, 7.7 %)                     
732 / 1000 in 909.76 / 1242.84 s -- reach,retrieve = (7.7 %, 7.7 %)                     
735 / 1000 in 912.66 / 1241.71 s -- reach,retrieve = (7.6 %, 7.6 %)                     
741 / 1000 in 914.75 / 1234.48 s -- reach,retrieve = (7.6 %, 7.6 %)                     
744 / 1000 in 916.31 / 1231.6 s -- reach,retrieve = (7.9 %, 7.9 %)                     
747 / 1000 in 919.46 / 1230.87 s -- reach,retrieve = (7.9 %, 7.9 %)                     
750 / 1000 in 920.9 / 1227.86 s -- reach,retrieve = (7.9 %, 7.9 %)                     
759 / 1000 in 925.35 / 12

183 / 1000 in 158.79 / 867.7 s -- reach,retrieve = (7.7 %, 7.7 %)                     
186 / 1000 in 160.73 / 864.16 s -- reach,retrieve = (7.5 %, 7.5 %)                     
189 / 1000 in 161.53 / 854.68 s -- reach,retrieve = (7.4 %, 7.4 %)                     
195 / 1000 in 163.6 / 838.99 s -- reach,retrieve = (7.2 %, 7.2 %)                     
201 / 1000 in 167.65 / 834.1 s -- reach,retrieve = (7.0 %, 7.0 %)                     
204 / 1000 in 168.42 / 825.58 s -- reach,retrieve = (6.9 %, 6.9 %)                     
210 / 1000 in 171.46 / 816.46 s -- reach,retrieve = (6.7 %, 6.7 %)                     
213 / 1000 in 173.85 / 816.22 s -- reach,retrieve = (6.6 %, 6.6 %)                     
219 / 1000 in 176.19 / 804.52 s -- reach,retrieve = (6.4 %, 6.4 %)                     
225 / 1000 in 180.26 / 801.14 s -- reach,retrieve = (6.2 %, 6.2 %)                     
234 / 1000 in 190.62 / 814.59 s -- reach,retrieve = (6.0 %, 6.0 %)                     
240 / 1000 in 254.52 / 1060.51 s --

678 / 1000 in 784.86 / 1157.62 s -- reach,retrieve = (6.9 %, 6.8 %)                     
687 / 1000 in 789.16 / 1148.71 s -- reach,retrieve = (6.8 %, 6.7 %)                     
693 / 1000 in 791.16 / 1141.65 s -- reach,retrieve = (6.8 %, 6.6 %)                     
702 / 1000 in 794.51 / 1131.78 s -- reach,retrieve = (6.7 %, 6.6 %)                     
708 / 1000 in 799.42 / 1129.12 s -- reach,retrieve = (6.6 %, 6.5 %)                     
714 / 1000 in 801.26 / 1122.21 s -- reach,retrieve = (6.6 %, 6.4 %)                     
723 / 1000 in 808.43 / 1118.16 s -- reach,retrieve = (6.5 %, 6.4 %)                     
726 / 1000 in 809.38 / 1114.85 s -- reach,retrieve = (6.5 %, 6.3 %)                     
732 / 1000 in 813.82 / 1111.78 s -- reach,retrieve = (6.6 %, 6.4 %)                     
738 / 1000 in 817.26 / 1107.4 s -- reach,retrieve = (6.5 %, 6.4 %)                     
741 / 1000 in 818.31 / 1104.34 s -- reach,retrieve = (6.5 %, 6.3 %)                     
744 / 1000 in 819.98 /

## ====================== deprecated ============================

## prepare cells

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
with gtimer.block("prepare_reference_cells_ltc_full"):
    ltc_full.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_effector"):
    ltc_effector.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_arm_05"):
    ltc_arm_05.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_arm_10"):
    ltc_arm_10.prepare_reference_cells(gscene)
print(gtimer)

In [ ]:
gscene.show_motion(Traj_reach)

## visualize

In [ ]:
vistem_list = []
for col_idx in grasp_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in grasp_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

In [ ]:
vistem_list = []
for col_idx in arm_10_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in arm_10_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

In [ ]:
vistem_list = []
for col_idx in full_scene_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_full.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in full_scene_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_full.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## grasp conversion

In [ ]:
gtimer.reset()
with gtimer.block("convert_vertices"):
    ltc_effector.convert_vertices(actor_vertinfo_list, HOME_DICT, Tref=Tref)
    ltc_effector.convert_vertices(object_vertinfo_list, HOME_DICT, Tref=Tref)
print(gtimer)

In [ ]:
vistem_list = []
for coll_idxes in ltc_effector.coll_idx_dict.values():
    for col_idx in coll_idxes:
        vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx]))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## scene conversion

In [ ]:
gtimer.reset()
with gtimer.block("ltc_arm_10_convert"):
    Tref_base = SE3(Tref[:3,:3], (0, 0, shoulder_height))
    ltc_arm_10.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
print(gtimer)

In [ ]:
vistem_list = []
for coll_idxes in ltc_arm_10.coll_idx_dict.values():
    for col_idx in coll_idxes:
        vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx]))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## remove object

In [ ]:

pscene.remove_object(obj_pscene.oname)
for handle in handles:
    gscene.remove(handle)

In [ ]:
gscene.get_items_on_links(TOOL_LINK_BUNDLE[0]) + gscene.get_items_on_links(TOOL_LINK_BUNDLE[1])

In [ ]:
with gtimer.block("convert"):
    ltc_full.convert([gtem for gtem in gscene if gtem not in gtems_robot], crob.home_dict)

In [ ]:
print(gtimer)

In [ ]:
gscene.show_pose(crob.home_pose)

In [ ]:
# for coll_idxes in latticizer.coll_idx_dict.values():
#     for col_idx in coll_idxes:
#         gscene.copy_from(latticizer.cell_refs[col_idx])

## deprecated python ver

In [ ]:
from pkg.utils.gjk import make_point3, get_point_list, get_point_list_list, get_point_list_list_from_point_list
from pkg.utils.gjk import get_gjk_distance, get_gjk_distance_min, get_gjk_distance_all

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
with gtimer.block("get_cell_vertices"):
    gscene_ref = GeometryScene(gscene.urdf_content, gscene.urdf_path, gscene.joint_names, gscene.link_names, rviz=False)
    centers = get_centers(Nwdh, L_CELL, OFFSET_ZERO).reshape((-1,3))
    cell_refs = []
    for icell, center in enumerate(centers):
        cell_refs.append(gscene_ref.create_safe(GEOTYPE.BOX, str(icell), "base_link", dims=(L_CELL,)*3, 
                                                center=center, rpy=(0,0,0), color=(1,1,1,0.2), 
                                                display=True, collision=False, fixed=True))
    cell_vertices = get_cell_vertices(centers, L_CELL)
    cell_vertices_gjk = get_point_list_list(cell_vertices)
    center_vertices_gjk = get_point_list_list(np.expand_dims(centers, axis=-2))

In [ ]:
print(gtimer)

In [ ]:
gtimer.reset()
sqrt3 = np.sqrt(3)
LCmax = L_CELL*sqrt3/2
Qdict = crob.home_dict
with gtimer.block("full"):
    for gtem in gscene:
#         if gtem in gtems_robot or not gtem.collision:
#             continue
        with gtimer.block("centers"):
            Tgtem = SE3_inv(gtem.get_tf(Qdict))
            centers_loc = np.abs(np.matmul(centers, Tgtem[:3,:3].transpose())+Tgtem[:3,3])
        if gtem.gtype == GEOTYPE.BOX and np.sum(np.abs(gtem.rpy))<1e-5:
            with gtimer.block("right box"):
                dist_list = np.max(centers_loc - np.divide(gtem.dims,2) - L_CELL, axis=-1)
                cell_idx_occupy = np.where(dist_list < 1e-3)[0]
                continue
        with gtimer.block("calc_center_dist"):
            dist_list = np.max(centers_loc - np.divide(gtem.dims,2) - LCmax, axis=-1)
            idx_candi = np.where(dist_list<0)[0]
        with gtimer.block("calc_points"):
            gtem_verts, gtem_radius = gtem.get_vertice_radius_from(Qdict)
        with gtimer.block("get_point_gjk"):
            gtem_verts_gjk = get_point_list(gtem_verts)
        with gtimer.block("get_distance_gjk"):
            cell_idx_occupy = []
            for idx in idx_candi:
                cell_candi = cell_vertices_gjk[idx]
                center_v_candi = center_vertices_gjk[idx]
                if ((get_gjk_distance(cell_candi, gtem_verts_gjk) - gtem_radius < 1e-4)):
                    cell_idx_occupy.append(idx)
#                 if ((get_gjk_distance(gtem_verts_gjk, center_v_candi) - gtem_radius - L_CELL/2 < 1e-4) or 
#                     (get_gjk_distance(cell_candi, gtem_verts_gjk) - gtem_radius < 1e-4)):
#                     cell_idx_occupy.append(idx)
print(gtimer)